In [ ]:
%pip install seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Let's load the file and make some checks

In [ ]:
df = pd.read_csv("churn.csv")

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df[df.duplicated()]

In [ ]:
df.columns
print('Number of unique values in each column:')
for label in df.columns:
        print(f'{label}: {df[label].nunique()}')

Let's drop the RowNumber, CustomerId, Surname columns as they are irrelevant to the analysis

In [ ]:
df.drop(columns=["RowNumber", "CustomerId", "Surname"], inplace=True)

In [ ]:
df.head()

The dataset is clean, let's see what columns are numerical vs categorical

In [ ]:
numerical_cols = df.select_dtypes(include=['number']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print("Numerical Columns:", numerical_cols)
print("Categorical Columns:", categorical_cols)

In [ ]:
sns.countplot(data=df, x='Exited', palette = "colorblind", hue = "Exited");

Let's show the percentages 

In [ ]:
ax = sns.countplot(x='Exited', data=df, palette='colorblind', hue='Exited')

# get the total count of the type column
total = df['Exited'].count()

# annotate the bars 
for c in ax.containers:
    ax.bar_label(c, fmt=lambda x: f'{(x/total)*100:0.1f}%')

plt.show()


In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(16, 10))

sns.countplot(data=df, x='Geography', hue='Exited', ax=ax[0][0])
ax[0][0].set_title('Geography and Churn')
sns.countplot(data=df, x='Gender', hue='Exited', ax=ax[0][1])
ax[0][1].set_title('Gender and Churn')
sns.countplot(data=df, x='Tenure', hue='Exited', ax=ax[0][2])
ax[0][2].set_title('Tenure amnd Churn')
sns.countplot(data=df, x='NumOfProducts', hue='Exited', ax=ax[1][0])
ax[1][0].set_title('Number of products and Churn')
sns.countplot(data=df, x='HasCrCard', hue='Exited', ax=ax[1][1])
ax[1][1].set_title('Credit Card and Churn')
sns.countplot(data=df, x='IsActiveMember', hue='Exited', ax=ax[1][2]);
ax[1][2].set_title('Activity and Churn')

People with a smaller amount of Products and people that are not active members are more likely to churn

Let's create some histograms for the numeric values with a bigger range 

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(16, 10))

sns.histplot(data=df, x='Balance', hue='Exited', ax=ax[0][0])
ax[0][0].set_title('Balance Distribution by Churn')
sns.histplot(data=df, x='Age', hue='Exited', ax=ax[0][1])
ax[0][1].set_title('Age Distribution by Churn')
sns.histplot(data=df, x='CreditScore', hue='Exited', ax=ax[1][0])
ax[1][0].set_title('Credit Score Distribution by Churn')
sns.histplot(data=df, x='EstimatedSalary', hue='Exited', ax=ax[1][1])
ax[1][1].set_title('Estimated Salary Distribution by Churn')

Customers with lower balance seem more likely to churn.
Age seems like a normal distribution as a percentage of churners with the center at 50 but most of the customers are under 40 years old.

In [ ]:
df = pd.get_dummies(df, columns=["Geography"])
df["Gender"] = df["Gender"].map({'Female': 0, 'Male': 1})

In [ ]:
df.head()

Since we have only three countries we can split them into three columns and make gender 0 for female and 1 for male

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Heatmap of the Data')
plt.show()

In [ ]:
corr_results = df.corrwith(df['Exited']).abs().sort_values(ascending=False)
corr_results

Age, IsActiveMember, Balance along with Gender and Geography seem to correlate.

Now let's try to do some feature engineering.

In [ ]:
#Create a new dataframe and keeping the old one just in case.
df_enriched = df.copy()

Let's put Age, Credit score, balance and estimated salary into groups.
We could define our own ranges and it would be better from a business perspective as each group would be more meaningful and not based on statistics. For example ages 18-28,26-36 etc instead of using quantile bins but we will keep it like this for now.

In [ ]:
df_enriched['Age_Groups'] = pd.qcut(df_enriched['Age'], 6, labels = [1, 2, 3, 4, 5, 6])
df_enriched["CreditsScore_Groups"] = pd.qcut(df_enriched['CreditScore'], 8, labels = [1, 2, 3, 4, 5, 6,7,8])
df_enriched["Balance_score_groups"] = pd.qcut(df_enriched['Balance'].rank(method="first"), 5, labels = [1, 2, 3, 4, 5]) #rank needed because of many 0s
df_enriched["EstSalaryScore_groups"] = pd.qcut(df_enriched['EstimatedSalary'], 10, labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

Let's create some more columns.

In [ ]:
df_enriched["BalanceSalaryRatio"] = df_enriched["Balance"] / (df_enriched["EstimatedSalary"] + 1)
df_enriched['Customer_Lifetime_Percentage'] = (df_enriched['Tenure'] / df_enriched['Age']) * 100
df_enriched["HighSalary"] = (df_enriched["EstimatedSalary"] > df_enriched["EstimatedSalary"].median()).astype(int)

In [ ]:
df_enriched.head()

We know we have an imbalanced dataset - most customers are not exiting.

In [ ]:
from sklearn.model_selection import train_test_split

X = df_enriched.drop('Exited', axis=1)
y = df_enriched['Exited']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

We could use class weights, like we do here, under sample the majority class or oversample the minority class.

Let's start training some models.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(class_weight='balanced', random_state=42)
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]
print(classification_report(y_test, y_pred))

roc_auc_rf = roc_auc_score(y_test, y_proba)
print("ROC-AUC:", roc_auc)

# ROC Curve
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, y_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr_rf, tpr_rf, label=f"ROC Curve (AUC = {roc_auc_rf:.3f})")
plt.plot([0, 1], [0, 1], linestyle='--') 
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Random Forest")
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(n_estimators=300,learning_rate=0.05,max_depth=3,random_state=42)
gb.fit(X_train, y_train)

In [ ]:
y_pred_gb = gb.predict(X_test)
y_proba_gb = gb.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred_gb))
roc_auc_gb = roc_auc_score(y_test, y_proba_gb)
print("ROC-AUC:", roc_auc_gb)

fpr_gb, tpr_gb, _ = roc_curve(y_test, y_proba_gb)

#ROC Curve
fpr_gb, tpr_gb, thresholds_gb = roc_curve(y_test, y_proba_gb)

plt.figure(figsize=(8, 6))
plt.plot(fpr_gb, tpr_gb, label=f"ROC Curve (AUC = {roc_auc_gb:.3f})")
plt.plot([0, 1], [0, 1], linestyle='--') 
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Gradient Boosting")
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr_rf, tpr_rf, label=f"Random Forest ROC (AUC={roc_auc_rf:.3f})")
plt.plot(fpr_gb, tpr_gb, label=f"Gradient Boosting ROC (AUC={roc_auc_gb:.3f})")
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison - Random Forest vs Gradient Boosting")
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Random Forest
fi_rf = pd.DataFrame({'Feature': X_train.columns, 'Importance': rf.feature_importances_}).sort_values('Importance', ascending=False)
# Gradient Boosting
fi_gb = pd.DataFrame({'Feature': X_train.columns, 'Importance': gb.feature_importances_}).sort_values('Importance', ascending=False)

print("=== Random Forest Feature Importance ===")
print(fi_rf.head(10))
print("\n=== Gradient Boosting Feature Importance ===")
print(fi_gb.head(10))

Some of the same features seem to be important for both models.

Let's change our threshold - we want to maximize recall and catch as many churners as possible. False positives will increase but are a lot less important than not catching a churner.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

def threshold_metrics(y_true, y_proba, thresholds):
    """
    Computes precision, recall, f1 for a range of probability thresholds.
    """
    results = []
    for t in thresholds:
        y_pred = (y_proba >= t).astype(int)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
        results.append((t, precision, recall, f1))
    return np.array(results)

# Define thresholds
thresholds = np.arange(0.1, 0.6, 0.05)

# Random Forest
rf_results = threshold_metrics(y_test, y_proba_rf, thresholds)

# Gradient Boosting
gb_results = threshold_metrics(y_test, y_proba_gb, thresholds)

# Display
print("=== Random Forest Threshold Optimization ===")
for t, p, r, f1 in rf_results:
    print(f"Threshold {t:.2f}: Precision={p:.3f}, Recall={r:.3f}, F1={f1:.3f}")

print("\n=== Gradient Boosting Threshold Optimization ===")
for t, p, r, f1 in gb_results:
    print(f"Threshold {t:.2f}: Precision={p:.3f}, Recall={r:.3f}, F1={f1:.3f}")


For both models the best threholds are at around 0.25 - 0.3

In [ ]:
best_threshold = 0.25 
y_pred_final = (y_proba_gb >= best_threshold).astype(int)

In [ ]:
gb_tuned = GradientBoostingClassifier(n_estimators=500,learning_rate=0.03,max_depth=5,subsample=0.9,max_features=0.8,random_state=42)
gb_tuned.fit(X_train, y_train)

In [ ]:
y_proba_gb_tuned = gb_tuned.predict_proba(X_test)[:, 1]
y_pred_gb_tuned = (y_proba_gb_tuned >= 0.25).astype(int)


print(classification_report(y_test, y_pred_gb_tuned))
roc_auc_gb_tuned = roc_auc_score(y_test, y_proba_gb_tuned)
print("ROC-AUC:", roc_auc_gb_tuned)

fpr_gb_tuned, tpr_gb_tuned, _ = roc_curve(y_test, y_proba_gb_tuned)

#ROC Curve
fpr_gb_tuned, tpr_gb_tuned, thresholds_gb_tuned = roc_curve(y_test, y_proba_gb_tuned)

plt.figure(figsize=(8, 6))
plt.plot(fpr_gb_tuned, tpr_gb_tuned, label=f"ROC Curve (AUC = {roc_auc_gb_tuned:.3f})")
plt.plot([0, 1], [0, 1], linestyle='--') 
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Gradient Boosting Tuned")
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()

Recall for churners has increased to 0.70 from 0.49 but precision has dropped. This is an acceptable trade-off since missing a churner is worse senario that falsely flagging one.

In [ ]:
import joblib

# Save Random Forest
joblib.dump(clf, "random_forest_churn.pkl")

# Save Gradient Boosting
joblib.dump(gb, "gradient_boosting_churn.pkl")

# Save Tuned Gradient Boosting
joblib.dump(gb_tuned, "gradient_boosting_tuned_churn.pkl")